In [2]:
import pyspark
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("bike").getOrCreate()
df = spark.read.option("header","true").csv("2017-fordgobike-tripdata.csv")

23/02/07 17:38:16 WARN Utils: Your hostname, murats-MacBook-Air.local resolves to a loopback address: 127.0.0.1; using 192.168.68.109 instead (on interface en0)
23/02/07 17:38:16 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/02/07 17:38:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
23/02/07 17:38:18 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


<ul>
<li>calculate the total distance for each bike and list the top 10</li>
<li>calculate the number of trips for each start station list top 10 and find the ratio of using as male or female</li>
<li>make a comparison to find the percentage of usage for customer and subscriber</li>
<li>calculate the age of all users and show the relation between the distance and the age</li>
</ul>

In [3]:
df.printSchema()

root
 |-- start_time: string (nullable = true)
 |-- end_time: string (nullable = true)
 |-- start_station_id: string (nullable = true)
 |-- start_station_name: string (nullable = true)
 |-- start_station_latitude: string (nullable = true)
 |-- start_station_longitude: string (nullable = true)
 |-- end_station_id: string (nullable = true)
 |-- end_station_name: string (nullable = true)
 |-- end_station_latitude: string (nullable = true)
 |-- end_station_longitude: string (nullable = true)
 |-- bike_id: string (nullable = true)
 |-- user_type: string (nullable = true)
 |-- member_birth_year: string (nullable = true)
 |-- member_gender: string (nullable = true)
 |-- pyment: string (nullable = true)



## calculate the number of trips for each start station list top 10 and find the ratio of using as male or female

In [4]:
#calculate the number of trips for each start station list top 10
trips_number = df.groupBy("start_station_id").count().orderBy("count", ascending=False)
trips_number.show(10)

+----------------+-----+
|start_station_id|count|
+----------------+-----+
|              15|15187|
|               6|13664|
|              30|12546|
|              67|12055|
|              58|11960|
|              21|11334|
|              81|10956|
|               3|10142|
|              22| 9926|
|              16| 9347|
+----------------+-----+
only showing top 10 rows



In [5]:
#find the ratio of using as male or female
df.select("member_gender").distinct().show()

+-------------+
|member_gender|
+-------------+
|         null|
|       Female|
|        Other|
|         Male|
+-------------+



In [10]:
top10_stations = [row[0] for row in trips_number.collect()[:10]]
topStationsDF = df.filter(
    (df['start_station_id'].isin(top10_stations)) 
    & (df['member_gender'].isin(["Female", "Male"]))
)
#topStationsDF.show()

In [19]:
from pyspark.sql.functions import round

byGender = topStationsDF.groupBy("start_station_id").pivot("member_gender").count()
byGender = byGender.withColumn("Rate", round(byGender["Female"]/byGender["Male"], 2))
byGender.show()

+----------------+------+----+----+
|start_station_id|Female|Male|Rate|
+----------------+------+----+----+
|              15|  2819|8586|0.33|
|               3|  1768|6070|0.29|
|              30|  2035|9813|0.21|
|              22|  1980|7512|0.26|
|              16|  1740|6532|0.27|
|               6|  1938|7444|0.26|
|              58|  1870|9005|0.21|
|              81|  2302|7773| 0.3|
|              67|  2105|9409|0.22|
|              21|  1631|8367|0.19|
+----------------+------+----+----+

